LOMBARDIA - ANALISI QUALITA' DELL'ARIA

FONTE INFORMAZIONI
https://dati.lombardia.it/stories/s/Meteo-inquinamento-aria-e-acqua-e-altri-dati-da-AR/auv9-c2sj/

INDICI QUALITA' DELL'ARIA
https://www.arpalombardia.it/Pages/Aria/Modellistica/Indice-qualit%C3%A0-aria.aspx

RILIEVI
https://www.dati.lombardia.it/Ambiente/Dati-stime-comunali/ysm5-jwrn /n
https://www.dati.lombardia.it/Ambiente/Dati-sensori-aria/nicp-bhqi

STAZIONI
https://www.dati.lombardia.it/Ambiente/Stazioni-qualit-dell-aria/ib47-atvt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

from sodapy import Socrata
client = Socrata("www.dati.lombardia.it", None)

#link_rilievi = 'https://www.dati.lombardia.it/resource/nicp-bhqi.csv'
results = client.get("nicp-bhqi",limit=10000000)
# Convert to pandas DataFrame
rl = pd.DataFrame.from_records(results)

link_stazioni = 'https://www.dati.lombardia.it/resource/ib47-atvt.csv'
st=pd.read_csv(link_stazioni)

#link_rilievi = 'https://www.dati.lombardia.it/resource/nicp-bhqi.csv'
#rl=pd.read_csv(link_rilievi)
#rl

In [ ]:
#st

In [ ]:
#CLEAN
rl['data']=pd.to_datetime(rl['data'])
rl['data']=rl['data'].dt.normalize()

#REMOVE NAN VALUE
rl=rl.replace('-9999',np.NaN)    
rl2=rl[rl['valore'].notna()]

#ADJUST DTYPES
rl2 = rl2.astype({"idsensore": int, "valore": float})
#rl2

In [ ]:
#FILTER STATION COLUMNS
stfi=st.loc[st['idsensore'].isin(rl2['idsensore']),['idsensore','nometiposensore','unitamisura','comune','location']]
stfi['location']=stfi['location'].str.strip('123.!? , \n\t')
#stfi

In [ ]:
#MERGE
rl2=rl2.merge(stfi,how='left')

In [ ]:
#rl2

In [ ]:
#FILTRO PER CITTA' (PAVIA)
rl3=rl2[rl2['comune']=='Pavia']
#rl3=rl2
#FILTRO DATA
#rl3=rl3[rl3['data']>='2020-09-01']
#rl3

In [ ]:
#check for null rows
missing_data = rl3.isnull()
#for column in missing_data.columns.values.tolist():
#    print(column)
#    print (missing_data[column].value_counts())
#    print("")

#drop null rows    
rl3.dropna(subset=["nometiposensore"], axis=0, inplace=True)
# reset index, because we droped two rows
rl3.reset_index(drop=True, inplace=True)

In [ ]:
#INDICI QUALITA' ARIA PM2.5,PM10, NO2, O3, SO2
#NOTA Monossido di Carbonio e' in mg/m³
rl3=rl3.replace('PM10 (SM2005)','PM10')
rl3=rl3.replace('Biossido di Zolfo','SO2 Biossido di Zolfo')
rl3=rl3.replace('Biossido di Azoto','NO2 Biossido di Azoto')
rl3=rl3.replace('Ozono','O3 Ozono')
rl3=rl3.replace('Particelle sospese PM2.5','PM2.5')

sensori=['NO2 Biossido di Azoto','SO2 Biossido di Zolfo','O3 Ozono','PM10','PM2.5']
rl4 = rl3[(rl3['nometiposensore'].str.contains('|'.join(sensori)))]
          
#rl4 = rl3[(rl3['nometiposensore'] == 'Biossido di Azoto') | (rl3['nometiposensore'] == 'Biossido di Zolfo')
#|(rl3['nometiposensore'] == 'Ozono')|(rl3['nometiposensore'] == 'PM10 (SM2005)')
#|(rl3['nometiposensore'] == 'Particelle sospese PM2.5')]

#rl4

In [ ]:
# GROUP DATA BY SENSOR
#rl4=rl3

rilievi = {}
for i in rl4['nometiposensore'].unique():
    rilievi[i] = rl4[rl4['nometiposensore'] == i]
    
    #raggruppa media settimanale e resetta indice
    rilievi[i]=rilievi[i].groupby(['nometiposensore', pd.Grouper(key='data', freq='W')])['valore'].mean().reset_index()
    #raggruppa media 3giorni e resetta indice
    #rilievi[i]=rilievi[i].groupby(['nometiposensore', pd.Grouper(key='data', freq='3D')])['valore'].mean().reset_index()

#rilievi

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

for i in rilievi:
    ax.plot(rilievi[i]['data'],rilievi[i]['valore'],label=i)

plt.title('Media settimanale Rilievi')
ax.set_xlabel('data')
ax.set_ylabel('densita µg/m³')
ax.legend()
plt.show()

IQA

In [ ]:
PM10=[500, 250, 100, 75, 50, 42, 35, 27, 20, 10]
PM25=[250, 75, 50, 40, 30, 25, 20, 15, 10, 5]

qlyscore=pd.DataFrame({'PM10':PM10,'PM25':PM25})
qlyscore

In [ ]:
plt.plot(qlyscore.index,qlyscore['PM25'])